### Coding Masked Self-Attention in PyTroch!!!

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Now Code a class that implements Masked Self Attention Mechanism.

In [ ]:
class MaskedSelfAttention(nn.Module):

    def __init__(self, d_model = 2,
                 row_dim = 0,
                 col_dim = 1):
        
        super().__init__()

        self.W_q = nn.Linear(in_features = d_model, 
                             out_features = d_model, 
                             bias = False)
        self.W_k = nn.Linear(in_features = d_model, 
                             out_features = d_model, 
                             bias = False)
        self.W_v = nn.Linear(in_features = d_model, 
                             out_features = d_model, 
                             bias = False) 
        self.row_dim = row_dim
        self.col_dim = col_dim

        

The forward method is where we actually calcuate the Masked Self-Attention values for each token. Just like before, we accept token encoding, but now also accpeting, Mask. etting default to None, we can use this to calcuate both MSA and SA both when needed.

If mask is not None: we add the mask to the scaled similarities with masked_fill(). To understyand how we add, imaging mask is a matrice of Trues and False. True is replaced with -1e9 and False is 0. Then we add this to scaled similarities and futher they poass thrugh Softmax for percentage similarity and then passes through V to calcuates attention score.

In [ ]:
def forward(self, token_encodings, mask = None):
    q = self.W_q(token_encodings)
    k = self.W_k(token_encodings)
    v = self.W_v(token_encodings)

    sims = torch.matmul(q,k.transpose(dim0 = self.row_dim, dim1 = self.col_dim))
    scaled_sims = sims / torch.tensor(k.size(self.col_dim) ** 0.5)

    if mask is not None:
        scaled_sims = scaled_sims.masked_fill(mask = mask, value = -1e9)

    attention_percents = F.softmax(scaled_sims, dim = self.col_dim)
    attention_scores = torch.matmul(attention_percents,v)
    

In [ ]:
## create a matrix of token encodings...
encodings_matrix = torch.tensor([[1.16, 0.23],
                                 [0.57, 1.36],
                                 [4.41, -2.16]])

torch.manual_seed(42)

# create a masked self attention object 

masked_self_attention = MaskedSelfAttention(d_model = 2,
                                            row_dim = 0,
                                            col_dim = 1)

# create a mask so that we don'e use tokens that comes after the token of interest
mask = torch.tril(torch.ones(3,3)) # Triangular Lower matrix of ones that is 1 below the diagonal and 0 above the diagonal.
mask = mask == 0 # Invert the mask so that the lower triangle is False, and the upper triangle is True. 
#This basically means that we are masking out the tokens that come after the token of interest.
# mask = mask == 0 is equivalent to mask = mask.bool() == False that is, we are converting the mask to a boolean tensor.


In [ ]:
masked_self_attention(encodings_matrix, mask) # this will return the attention scores for each token in the encodings matrix.
# The attention scores are the weighted sum of the values of the tokens in the encodings matrix.